In [69]:
import numpy as np, pandas as pd
from ISLP import load_data
import seaborn as sns
from matplotlib import pyplot as plt
import ISLP
from ISLP import load_data
import itertools
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import roc_curve
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError ,
    R2Score)
from scipy.stats import ttest_ind
from sklearn.metrics import f1_score
from torchinfo import summary
from torchvision.io import read_image
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
import pytorch_lightning as L
from pytorch_lightning.loggers import CSVLogger
from torchvision.datasets import MNIST , CIFAR100
from IPython.display import clear_output
from torchvision.models import (resnet50 ,
    ResNet50_Weights)
from torchvision.transforms import (Resize ,
    Normalize ,
    CenterCrop ,
    ToTensor)
from ISLP.torch import (SimpleDataModule ,
    SimpleModule ,
    ErrorTracker ,
    rec_num_workers)
from ISLP.torch.imdb import (load_lookup ,
    load_tensor ,
    load_sparse ,
    load_sequential)
from glob import glob
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import statsmodels.api as sm
import json
sns.set_palette("pastel")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [70]:
Carseats = load_data('Carseats')

In [71]:
df = Carseats.drop(columns=['Urban', 'US', 'ShelveLoc'])
X = df.drop('Sales', axis=1)
y = df['Sales']

In [72]:
p_val = {}
for i in X.columns:
    X_i = pd.DataFrame(X[i])
    X_i['intercept'] = 1
    model = sm.OLS(y, X_i)
    results = model.fit()
    p_val[i] = [results.pvalues[i]]
p_val = pd.DataFrame(p_val).T

In [85]:
# Raw Pval

p_val

,0
CompPrice,2.009398e-01
Income,2.309670e-03
Advertising,4.377677e-08
Population,3.139816e-01
Price,7.618187e-21
Age,2.788950e-06
Education,2.999442e-01


In [86]:
# Let's use the standard p_value theshold approach
alpha = 0.05
p_val[p_val[0] < alpha]

,0
Income,2.309670e-03
Advertising,4.377677e-08
Price,7.618187e-21
Age,2.788950e-06


In [87]:
# Let's use Homm approach :

ordered_pvalues = p_val[0].sort_values()
alpha = 0.05
m = len(ordered_pvalues)
for i in range(m):
    if ordered_pvalues.iloc[i] > (alpha/(m+1-(i+1))):
        break
p_val.loc[ordered_pvalues[:i].index]

,0
Price,7.618187e-21
Advertising,4.377677e-08
Age,2.788950e-06
Income,2.309670e-03


In [88]:
# Benjamini–Hochberg 

q = 0.2

ordered_pvalues = p_val[0].sort_values()
m = len(ordered_pvalues)
largest_i = 0
for i in range(m):
    if ordered_pvalues.iloc[i] < (q*(i+1))/m:
        largest_i = i
p_val.loc[ordered_pvalues[:largest_i+1].index]

,0
Price,7.618187e-21
Advertising,4.377677e-08
Age,2.788950e-06
Income,2.309670e-03
